In [1]:
import math
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
vavel_header_raw = "versionID;line;brigade;time;lon;lat;rawLon;rawLat;status;delay;delayAtStop;plannedLeaveTime;nearestStop;nearestStopDistance;nearestStopLon;nearestStopLat;previousStop;previousStopLon;previousStopLat;previousStopDistance;previousStopArrivalTime;previousStopLeaveTime;nextStop;nextStopLon;nextStopLat;nextStopDistance;nextStopTimetableVisitTime;courseIdentifier;courseDirection;timetableIdentifier;timetableStatus;receivedTime;processingFinishedTime;onWayToDepot;overlapsWithNextBrigade;overlapsWithNextBrigadeStopLineBrigade;atStop;speed;oldDelay;serverId;delayAtStopStopSequence;previousStopStopSequence;nextStopStopSequence;delayAtStopStopId;previousStopStopId;nextStopStopId"
vavel_header = vavel_header_raw.split(";")

In [4]:
paths = [
    r'C:\Users\kosma\Desktop\MAGISTER\VAVEL\2018-05-23',
    r'C:\Users\kosma\Desktop\MAGISTER\VAVEL\2018-05-24',
    r'C:\Users\kosma\Desktop\MAGISTER\VAVEL\2018-05-25',
] # use your path

In [5]:
import glob


li = []
all_files = []
for path in paths:
    all_files += glob.glob(path + "/*")
    
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, names=vavel_header, sep = ';')
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
li = []

c:\users\kosma\desktop\magister\featextream\venv\lib\site-packages\IPython\core\interactiveshell.py:3170: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
frame.count()

versionID                                 3679158
line                                      3679158
brigade                                   3679158
time                                      3679158
lon                                       3679158
lat                                       3679158
rawLon                                    3679158
rawLat                                    3679158
status                                    3679158
delay                                     3679158
delayAtStop                               3590452
plannedLeaveTime                          3590452
nearestStop                               3592128
nearestStopDistance                       3679158
nearestStopLon                            3679158
nearestStopLat                            3679158
previousStop                              3590945
previousStopLon                           3679158
previousStopLat                           3679158
previousStopDistance                      3679158


In [4]:
#  frame = pd.read_csv(r'C:\Users\kosma\Desktop\MAGISTER\VAVEL\2018-05-23\part-0-0', index_col=None, names=vavel_header, sep = ';')

In [11]:
drop_columns = ['versionID', 'rawLon', 'rawLat', 'receivedTime', 'processingFinishedTime','overlapsWithNextBrigadeStopLineBrigade', 'onWayToDepot', 'timetableIdentifier', 'serverId']
time_columns = ['time','plannedLeaveTime', 'previousStopArrivalTime', 'previousStopLeaveTime', 'nextStopTimetableVisitTime']

In [12]:
frame.drop(columns = drop_columns, inplace=True)

In [13]:
for col in time_columns:
    frame[col] = pd.to_numeric(pd.to_datetime(frame[col], errors='coerce')) // 10**9

In [10]:
# frame[frame['courseIdentifier'] == '9_12_218_2403'][['time', 'previousStopArrivalTime', 'previousStopLeaveTime','nextStopTimetableVisitTime', 'delayAtStopStopSequence','previousStopStopSequence','nextStopStopSequence','delay', 'oldDelay']].to_csv('analysis.csv')

In [11]:
# frame[(frame['courseIdentifier'] ==  '9_021_218_2343') |  (frame['courseIdentifier'] == '9_021_218_2446')][['courseIdentifier','time', 'previousStopArrivalTime', 'previousStopLeaveTime','nextStopTimetableVisitTime', 'delayAtStopStopSequence','previousStopStopSequence','nextStopStopSequence','delay', 'oldDelay']].to_csv('tworides.csv')

In [14]:
polishLetterColumns  = ['delayAtStop','nearestStop','previousStop','nextStop','courseDirection']
floorColumns = ['status','courseIdentifier','delayAtStopStopId','previousStopStopId','nextStopStopId']
transColumns = polishLetterColumns + floorColumns

In [15]:
translateFromChars = 'ąćęłńóśźż_"'
translateToChars = 'acelnoszz- '
transTable = str.maketrans(translateFromChars,translateToChars)

In [16]:
for column in transColumns:
    frame[column] = frame[column].str.lower().str.translate(transTable)

In [17]:
frame["id1"]  = frame["line"].astype(str) + "--" + frame["brigade"].astype(str)

In [31]:
slices = 25
eps = 1e-4
max_lon = 21.1203908 #frame['lon'].max() + eps
min_lon = 20.9059783 #frame['lon'].min() - eps
max_lat = 52.3344658 #frame['lat'].max() + eps
min_lat = 52.1755439 #frame['lat'].min() - eps

lon_space = [0] + list(np.linspace(min_lon,max_lon,slices+1)) + [360]
lat_space = [0] + list(np.linspace(min_lat,max_lat,slices+1)) + [360]


In [34]:
quadrant_df = pd.DataFrame(data= {
    'lonMin': lon_space[:-1], 'lonMax': lon_space[1:],
    'latMin': lat_space[:-1], 'latMax': lat_space[1:],
    'id': range(0,slices + 2)
})

In [36]:
intervals_lon = quadrant_df.set_index(pd.IntervalIndex.from_arrays(quadrant_df['lonMin'], 
                                                             quadrant_df['lonMax']))['id']
intervals_lat = quadrant_df.set_index(pd.IntervalIndex.from_arrays(quadrant_df['latMin'], 
                                                             quadrant_df['latMax']))['id']
frame["lonSlice"]  = frame['lon'].map(intervals_lon)
frame["latSlice"]  = frame['lat'].map(intervals_lat)

In [37]:


frame["quadrant"] = frame["lonSlice"].astype(str) + "--" + frame["latSlice"].astype(str)

In [38]:
frame["labelId"] = frame["courseIdentifier"].astype(str) + "--" + frame["nextStopStopSequence"].astype(str)

In [39]:
partitionKey = ['courseIdentifier','nextStopStopSequence']


frame['rowNo'] = frame.sort_values(['time'], ascending=False)\
             .groupby(partitionKey)\
             .cumcount() + 1



In [40]:
frame = frame.fillna(-1)

In [41]:
frame['label'] = np.nan

frame['label'] = frame[frame['rowNo'] == 1].sort_values(by=['time'], ascending=True)\
                       .groupby(['courseIdentifier'])['oldDelay'].shift(-1)


In [24]:
# frame[(frame['courseIdentifier'] ==  '9_021_218_2343') |  (frame['courseIdentifier'] == '9_021_218_2446')][['courseIdentifier','time', 'previousStopArrivalTime', 'nextStopTimetableVisitTime', 'delayAtStopStopSequence','previousStopStopSequence','nextStopStopSequence','delay', 'oldDelay','rowNo','label']].to_csv('tworides.csv')

In [42]:
frame.to_csv('result.csv', sep = ';', na_rep='?', index=False)

In [43]:
frame[~frame['label'].isna()].to_csv('result_sup.csv', sep = ';', na_rep='?', index=False)

In [44]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3679158 entries, 0 to 3679157
Data columns (total 44 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   line                        int64  
 1   brigade                     object 
 2   time                        int64  
 3   lon                         float64
 4   lat                         float64
 5   status                      object 
 6   delay                       float64
 7   delayAtStop                 object 
 8   plannedLeaveTime            int64  
 9   nearestStop                 object 
 10  nearestStopDistance         float64
 11  nearestStopLon              float64
 12  nearestStopLat              float64
 13  previousStop                object 
 14  previousStopLon             float64
 15  previousStopLat             float64
 16  previousStopDistance        float64
 17  previousStopArrivalTime     int64  
 18  previousStopLeaveTime       int64  
 19  nextStop             

In [46]:
frame[~frame['label'].isna()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 303864 entries, 159 to 3679105
Data columns (total 44 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   line                        303864 non-null  int64  
 1   brigade                     303864 non-null  object 
 2   time                        303864 non-null  int64  
 3   lon                         303864 non-null  float64
 4   lat                         303864 non-null  float64
 5   status                      303864 non-null  object 
 6   delay                       303864 non-null  float64
 7   delayAtStop                 303864 non-null  object 
 8   plannedLeaveTime            303864 non-null  int64  
 9   nearestStop                 303864 non-null  object 
 10  nearestStopDistance         303864 non-null  float64
 11  nearestStopLon              303864 non-null  float64
 12  nearestStopLat              303864 non-null  float64
 13  previousSto

In [4]:
frame = pd.read_csv('result_sup.csv', index_col=None, sep = ';')

In [7]:
drop_columns = ['line','brigade','delayAtStop','nearestStop',
                'nearestStopDistance','nearestStopLon','nearestStopLat','previousStop', 'previousStopLon', 'previousStopLat',
               'previousStopArrivalTime', 'nextStop', 'nextStopLon', 'nextStopLat', 'courseDirection','overlapsWithNextBrigade',
               'atStop','oldDelay', 'delayAtStopStopSequence','previousStopStopSequence','nextStopStopSequence',
               'delayAtStopStopId', 'previousStopStopId', 'nextStopStopId', 'lonSlice', 'latSlice', 'rowNo']
frame_slim = frame.drop(columns=drop_columns)

In [10]:
frame_slim[frame_slim['timetableStatus']!= "SAFE"]

,time,lon,lat,status,delay,plannedLeaveTime,previousStopDistance,previousStopLeaveTime,nextStopDistance,nextStopTimetableVisitTime,courseIdentifier,timetableStatus,speed,id1,quadrant,labelId,label
14,1527033864,21.047392,52.247852,moving,0.0,1527033840,351.589339,1527033843,78.106107,1527033840,26-6-126-2355,UNSAFE,1.58,26--6,17--12,26-6-126-2355--9.0,0.0
45,1527034971,20.978571,52.236389,moving,0.0,1527034980,465.475888,-9223372037,49.298441,1527034980,24-11-121-2350,UNSAFE,2.32,24--11,9--10,24-11-121-2350--24.0,39.0
62,1527035483,21.119535,52.238224,moving,0.0,1527035400,49.277509,-9223372037,44.775775,1527035400,9-021-218-2343,UNSAFE,3.01,9--21,25--10,9-021-218-2343--36.0,0.0
75,1527036373,21.118382,52.238277,moving,0.0,1527036360,36.277961,1527036373,31.937305,1527036360,9-021-218-2446,UNSAFE,3.66,9--21,25--10,9-021-218-2446--3.0,29.0
76,1527036435,21.114327,52.239945,moving,29.0,1527036420,300.635267,1527036389,75.097609,1527036420,9-021-218-2446,UNSAFE,2.20,9--21,25--11,9-021-218-2446--4.0,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303714,1527292312,21.114361,52.239967,moving,52.0,1527292260,300.123249,1527292239,76.595629,1527292260,24-11-121-2350,UNSAFE,4.24,24--11,25--11,24-11-121-2350--3.0,71.0
303772,1527292470,21.008724,52.220146,moving,0.0,1527292320,308.315313,1527292463,172.593985,1527292320,15-2-48-2330,UNSAFE,10.44,15--2,12--8,15-2-48-2330--19.0,165.0
303774,1527292478,21.022684,52.206089,moving,0.0,1527292260,362.304323,1527292458,124.324112,1527292260,4-12-152-2310,UNSAFE,18.00,4--12,14--5,4-12-152-2310--30.0,234.0
303803,1527292573,20.913771,52.252209,moving,0.0,1527292200,655.445031,1527292459,85.664086,1527292200,24-017-121-2300,UNSAFE,4.81,24--17,1--13,24-017-121-2300--37.0,0.0


In [11]:
frame_slim.to_csv('result_sup_slim.csv', sep = ';', na_rep='?', index=False)

In [3]:
frame_slim = pd.read_csv('result_sup_slim.csv', index_col=None, sep = ';')

In [5]:
frame_slim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303864 entries, 0 to 303863
Data columns (total 17 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   time                        303864 non-null  int64  
 1   lon                         303864 non-null  float64
 2   lat                         303864 non-null  float64
 3   status                      303864 non-null  object 
 4   delay                       303864 non-null  float64
 5   plannedLeaveTime            303864 non-null  int64  
 6   previousStopDistance        303864 non-null  float64
 7   previousStopLeaveTime       303864 non-null  int64  
 8   nextStopDistance            303864 non-null  float64
 9   nextStopTimetableVisitTime  303864 non-null  int64  
 10  courseIdentifier            303864 non-null  object 
 11  timetableStatus             303864 non-null  object 
 12  speed                       303864 non-null  float64
 13  id1           